In [1]:
import ast
import unicodedata
import numpy as np
import pandas as pd

from tqdm import tqdm
from utils_data_wrangling import *

In [2]:
car_features = pd.DataFrame()
failed_urls = pd.DataFrame()

for i in [j for j in range(500, 29500, 500)] + ['final']:
    aux_features = pd.read_csv(f'data/partial_data/cars_features_{i}.csv')
    aux_failed = pd.read_csv(f'data/partial_data/failed_urls_{i}.csv')

    car_features = pd.concat([car_features, aux_features], axis=0)
    failed_urls = pd.concat([failed_urls, aux_failed], axis=0)

car_features = car_features.rename(columns={'other_fatures': 'other_features'})

In [3]:
failed_urls['error'].value_counts()

error
'NoneType' object has no attribute 'p'    1978
Name: count, dtype: int64

### Get possible values for feature cols

In [4]:
car_features['maker'] = car_features['maker'].str.lower()
car_features['model'] = car_features['model'].str.lower()
car_features['other_features'] = car_features['other_features'].str.lower()
car_features['city'] = car_features['city'].str.lower()
car_features['price'] = car_features['price'].replace({
    'R\$': '',  # Remove R$
    '\s': '',   # Remove spaces
    '\.': ''    # Remove dots
}, regex=True).astype(float)
car_features['year'] = np.where(car_features['year'] == 2924, 2024, car_features['year'])
car_features['engine'] = np.where(car_features['engine'] == '1.6/s', '1.6', car_features['engine'])

# maker - ok
# model - ok
years = [str(int(i)) for i in range(1950, 2026)] + [str(float(i)) for i in range(1950, 2026)]
engines = list(
    car_features
    .query("engine not in ('Cj-5', 'Cj-3', 'Elétrico', '2p', 'B12p', 'Ev', '44,9', 'P6', '82,5', 'P8', 'Se')")
    .astype({'engine': float})
    .groupby('engine', as_index=False)
    [['id']]
    .count()
    .sort_values('engine', ascending=True)
    .query("engine < 27")
    ['engine']
) + ['Cj-5', 'Cj-3', 'Elétrico', 'Ev', 'P6', 'P8', '1.050']
engines = [str(i) for i in engines]
transmissions = ['Manual', 'Automático', 'Automático CVT', 'Automatizado', '100% Elétrico', 'Alcool', 'Elétrico']
fuel_types = ['Flex', 'Gasolina', 'Diesel', 'Alcool', 'Híbrido e Flex', 'Híbrido e Gasolina',
              'Elétrico', 'GNV e Gasolina', 'GNV e Flex', 'GNV'] 
body_types = ['Hatch', 'SUV', 'Sedan', 'Picape', 'Monovolume', 'SW-Perua', 'Van', 
              'Furgão Pequeno', 'Minivan', 'Coupé-Fast Back', 'Conversível']
colors = (
    car_features
    .groupby('color', as_index=False)
    [['id']]
    .count()
    .sort_values('id', ascending=False)
    [lambda x: ~x['color'].str.contains(r'\d+\.?\d*\s*KM', na=False)]
    ['color']
) 
colors = [i for i in colors]
# mileage - ok
# doors e valves são meio inuteis na real, o que mais tem nessas colunas são infos de modelo 
# do carro, tipo civic ls, prisma joy e afins

### Get the correct info for each feature col

In [5]:
car_features = (
    car_features
    .pipe(get_spread_info, 'year', years)
    .apply(lambda x: x.astype(str))
    .pipe(get_spread_info, 'engine', engines)
    .pipe(get_spread_info, 'transmission', transmissions)
    .pipe(get_spread_info, 'fuel_type', fuel_types)
    .pipe(get_spread_info, 'body_type', body_types)
    .pipe(get_spread_info, 'color', colors)
    .pipe(get_spread_mileage_info)
    .astype({'mileage': str, 'year': str})
    .assign(year = lambda x: np.where(~x['year'].isin(years), np.nan, x['year']))
    .assign(engine = lambda x: np.where(~x['engine'].isin(engines), np.nan, x['engine']))
    .assign(transmission = lambda x: np.where(~x['transmission'].isin(transmissions), np.nan, x['transmission']))
    .assign(fuel_type = lambda x: np.where(~x['fuel_type'].isin(fuel_types), np.nan, x['fuel_type']))
    .assign(body_type = lambda x: np.where(~x['body_type'].isin(body_types), np.nan, x['body_type']))
    .assign(color = lambda x: np.where(~x['color'].isin(colors), np.nan, x['color']))
    .assign(mileage = lambda x: np.where(~x['mileage'].str.endswith(('KM', "KM KM")), np.nan, x['mileage']))
    .assign(year = lambda x: np.where(~x['spread_year'].isnull(), x['spread_year'], x['year']))
    .assign(engine = lambda x: np.where(~x['spread_engine'].isnull(), x['spread_engine'], x['engine']))
    .assign(transmission = lambda x: np.where(~x['spread_transmission'].isnull(), x['spread_transmission'], x['transmission']))
    .assign(fuel_type = lambda x: np.where(~x['spread_fuel_type'].isnull(), x['spread_fuel_type'], x['fuel_type']))
    .assign(body_type = lambda x: np.where(~x['spread_body_type'].isnull(), x['spread_body_type'], x['body_type']))
    .assign(color = lambda x: np.where(~x['spread_color'].isnull(), x['spread_color'], x['color']))
    .assign(mileage = lambda x: np.where(~x['spread_mileage'].isnull(), x['spread_mileage'], x['mileage']))
    .drop(['spread_year', 'spread_engine', 'spread_transmission', 'spread_fuel_type', 'spread_body_type', 'spread_color', 'spread_mileage'], axis=1)
)

## Working with other_features col

### Making them usable

In [6]:
extra_features = []
for item in [sublista for sublista in car_features['other_features']]:
    if item != '[]':
        try:
            lista_convertida = ast.literal_eval(item)
            extra_features.extend(lista_convertida)
        except:
            continue

extra_features = [fix_string(item) for item in extra_features]
extra_features = list(
    pd.DataFrame({'item': extra_features})
    .assign(um = 1)
    .groupby('item', as_index=False)
    [['um']]
    .sum()
    .sort_values('um', ascending=False)
    .head(95)
    ['item']
)

### Creating a col for each extra features and populating them

In [7]:
for feature in extra_features:
    car_features[feature] = np.nan

car_features['other_features'] = car_features.apply(lambda x: fix_items(x), axis=1)

for feature in tqdm(extra_features):
    car_features[feature] = car_features.apply(lambda x: check_feature(x, feature), axis=1)

100%|██████████| 95/95 [01:44<00:00,  1.10s/it]


In [9]:
car_features.to_csv('data/car_features.csv', index=False)